In [1]:
import random
from sklearn.cross_validation import KFold
import csv
from deap import base, creator, tools, algorithms
import numpy

#make fitness and individual classes
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

#get dataset
with open("../../3ormoreDATLOW.csv",'r') as f:
    it = csv.reader(f)
    listit = list(it)

#number of individuals
POPNUM = 10
#number of generations
GENNUM = 10000    
#number of features
FEATNUM = len(listit[0])-2
#number of folds
numfolds = 10

#split data from header
header = listit[0]
listit = listit[1:]

#shuffle the data
random.shuffle(listit)

#find testing index
kf = KFold(len(listit),n_folds=numfolds,random_state=True)
indices = []
for trainind,testind in kf:
    indices.append(testind)

listit

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[['TCGA-AG-3887',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '-1'],
 ['TCGA-AG-3605',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '-1'],
 ['TCGA-AF-3400',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '-1'],
 ['TCGA-AG-3882',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '-1'],
 ['TCGA-AA-3858',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '-1'],
 ['TCGA-AA-A00K',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '-1'],
 ['TCGA-AA-3844',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
 

In [2]:
def evalOneMax(individual,trainset):
    sumscore = 0
    capturedTP = []
    capturedFP = []
    for y in range (0,len(individual)):
        gotone = False
        if(individual[y] == 1):
            #score up this feature
            for x in range(1,len(trainset)):
                if(float(trainset[x][y+1]) == 1.0):
                    if(float(trainset[x][len(trainset[1])-1]) == 1.0):
                        if trainset[x][0] not in capturedTP:
                            capturedTP.append(trainset[x][0])
                            sumscore += 1
                            gotone = True
                    elif(float(trainset[x][len(trainset[1])-1]) == -1.0):
                        if trainset[x][0] not in capturedFP:
                            capturedFP.append(trainset[x][0])
                            sumscore -= 1
            if(gotone == False):
                sumscore -= 1
    return sumscore,

In [3]:
#looking at all features overall
def runGreedyOnTest(model, testdata):
    numrows = len(testdata)
    numcols = len(testdata[0])-1
    results = {}
    for x in range(1, numrows):
        for y in range(1, numcols): #program won't be able to see background/foreground
            if (float(testdata[x][y]) == 1.0):
                if (testdata[0][y] in model):
                    results[testdata[x][0]] = 1.0
    for x in range(1,numrows):
        if(testdata[x][0] not in results):
            results[testdata[x][0]] = -1.0
    return results
def evalResults(dictofseqs,givendat,notpicked):
    falsepos = falseneg = truepos = trueneg = lowmiss = lowoccur = 0
    for x in range (1,len(givendat)):
        if(givendat[x][0] in dictofseqs):
            if(float(dictofseqs[givendat[x][0]]) == 1.0):
                if(float(givendat[x][len(givendat[0])-1]) == -1.0):
                    falsepos += 1
                elif(float(givendat[x][len(givendat[0])-1]) == 2.0):
                    lowoccur += 1
                else:
                    truepos += 1
                    if givendat[x][0] not in notpicked:
                        notpicked.append(givendat[x][0])
            else:
                if(float(givendat[x][len(givendat[0])-1]) == 1.0):
                    falseneg += 1
                elif(float(givendat[x][len(givendat[0])-1]) == 2.0):
                    lowmiss += 1
                else:
                    trueneg += 1
    tot = len(dictofseqs)
    overallerror = (falseneg+falsepos)/(trueneg+falseneg+falsepos+truepos)
    sens = truepos/(truepos+falseneg)
    spec = trueneg/(trueneg+falsepos)
    print("False Positives: " + str(falsepos))
    print("False Negatives: " + str(falseneg))
    print("True Positives: " + str(truepos))
    print("True Negatives: " + str(trueneg))
    print("CIMP-Low Occurrences: " + str(lowoccur))
    print("CIMP-Low Not Covered: " + str(lowmiss))
    print("Proportion of false positives: " + str((falsepos/tot)))
    print("Proportion of false negatives:" + str((falseneg/tot)))
    print("Accuracy: " + str((1-overallerror)))
    print("Error rate: " + str(overallerror))
    print("Sensitivity: " + str(sens))
    print("Specificity: " + str(spec))



In [4]:
def makegen(trainset):
    #register tools
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=FEATNUM)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", evalOneMax, trainset)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.10)
    toolbox.register("select", tools.selTournament, tournsize=3)
    
    pop = toolbox.population(n=POPNUM)
    hof = tools.HallOfFame(5)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=GENNUM, stats=stats, halloffame=hof, verbose=False)
    
    return pop, logbook, hof


for i in range(numfolds):
    #make testing and training sets
    trainset = [header]
    testset = [header]
    for x in range(0,len(listit)):
        if(x in indices[i]):
            testset.append(listit[x])
        else:
            trainset.append(listit[x])
    
    #setup GA
    pop,logbook,hof = makegen(trainset)
    model = []
    for x in range(0, len(hof[0])):
        if(float(hof[0][x]) == 1.0):
            model.append(trainset[0][x+1])
    '''print("Best individual is: %s\nwith fitness: %s" % (hof[0], hof[0].fitness))for x in range(0,1):
    print("Individual ranked #" + str(x+1))
    print(hof[x])
    print("with fitness: " + str(hof[x].fitness) + "\n")'''
    print(model)
    print("-----------------------------------")
    notpicked = []
    dictofseqs = runGreedyOnTest(model,testset)
    evalResults(dictofseqs,testset,notpicked)
    print("-----------------------------------")

['MSH6_GRCh37_2:48030640-48030647_Frame-Shift-Del-DEL-CCCCCCCC----', 'HMMR_GRCh37_5:162917426-162917434_In-Frame-Del-DEL-AAAAAAAAA----', 'ACVR2A_GRCh37_2:148683686-148683693_Frame-Shift-Del-DEL-AAAAAAAA----', 'CASP5_GRCh37_11:104878041-104878050_Frame-Shift-Del-DEL-TTTTTTTTTT----', 'MSH3_GRCh37_5:79970915-79970922_Frame-Shift-Del-DEL-AAAAAAAA----']
-----------------------------------
False Positives: 0
False Negatives: 1
True Positives: 2
True Negatives: 20
CIMP-Low Occurrences: 0
CIMP-Low Not Covered: 0
Proportion of false positives: 0.0
Proportion of false negatives:0.043478260869565216
Accuracy: 0.9565217391304348
Error rate: 0.043478260869565216
Sensitivity: 0.6666666666666666
Specificity: 1.0
-----------------------------------
['MSH6_GRCh37_2:48030640-48030647_Frame-Shift-Del-DEL-CCCCCCCC----', 'PRDM2_GRCh37_1:14108749-14108757_In-Frame-Del-DEL-AAAAAAAAA----', 'KRAS_GRCh37_12:25398284-25398284_Missense-Mutation-SNP-C-T-T', 'CASP5_GRCh37_11:104878041-104878050_Frame-Shift-Del-DEL-

['BRAF_GRCh37_7:140453136-140453136_Missense-Mutation-SNP-A-A-T_Missense-Mutation-SNP-A-T-T', 'GRB14_GRCh37_2:165365288-165365296_In-Frame-Del-DEL-TTTTTTTTT----', 'MBD4_GRCh37_3:129155548-129155557_Frame-Shift-Del-DEL-TTTTTTTTTT----', 'HMMR_GRCh37_5:162917426-162917434_In-Frame-Del-DEL-AAAAAAAAA----', 'PIK3CA_GRCh37_3:178952085-178952085_Missense-Mutation-SNP-A-G-G', 'ACVR2A_GRCh37_2:148683686-148683693_Frame-Shift-Del-DEL-AAAAAAAA----', 'CASP5_GRCh37_11:104878041-104878050_Frame-Shift-Del-DEL-TTTTTTTTTT----', 'ATR_GRCh37_3:142274740-142274749_Frame-Shift-Del-DEL-TTTTTTTTTT----', 'AIM2_GRCh37_1:159032487-159032496_Frame-Shift-Del-DEL-TTTTTTTTTT----']
-----------------------------------
False Positives: 0
False Negatives: 1
True Positives: 1
True Negatives: 20
CIMP-Low Occurrences: 0
CIMP-Low Not Covered: 0
Proportion of false positives: 0.0
Proportion of false negatives:0.045454545454545456
Accuracy: 0.9545454545454546
Error rate: 0.045454545454545456
Sensitivity: 0.5
Specificity: 1.0
